In [176]:
#profitableHospital-project

In [177]:
#dependencies
import pandas as pd


import requests
import json
from config import gkey


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

import joblib

In [178]:
#initial file read
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2017.csv")
#df = df.dropna(axis='columns', how='all')
#df = df.dropna()
df.head()

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (105) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,4492364.0,2638799.0,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,31174.0,-1114660.0,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095
2,450178,PECOS COUNTY MEMORIAL HOSPITAL,386 W IH-10,FORT STOCKTON,TX,79735,PECOS,99945.0,R,STH,...,2096862.0,576278.0,NaN,576278.0,0.455741,719760.0,1275211.0,NaN,NaN,601392
3,341317,PIONEER HLTH SERV OF STOKES CNTY,1570 NC & 89 HWY,DANBURY,NC,27016,STOKES,49180.0,U,CAH,...,1542201.0,640268.0,622592.0,17676.0,1.154767,578366.0,801800.0,159516.0,702117.0,601721
4,110003,MAYO CLINIC HEALTH SYSTEM WAYCROSS,1900 TEBEAU STREET,WAYCROSS,GA,31501,WARE,99911.0,R,STH,...,224757.0,-6170184.0,NaN,-6170184.0,0.271734,3113423.0,11231324.0,1331.0,8211.0,603846


In [179]:
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head()

,State Code,CON
0,AK,1
1,AL,1
2,AR,1
3,AZ,0
4,CA,0


In [180]:
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,2


In [181]:
mainFile1 = pd.merge(df, conTable, how= "left", on=['State Code'])
mainFile1.head()

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,CON
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,2638799.0,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181,1.0
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,-1114660.0,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095,1.0
2,450178,PECOS COUNTY MEMORIAL HOSPITAL,386 W IH-10,FORT STOCKTON,TX,79735,PECOS,99945.0,R,STH,...,576278.0,NaN,576278.0,0.455741,719760.0,1275211.0,NaN,NaN,601392,0.0
3,341317,PIONEER HLTH SERV OF STOKES CNTY,1570 NC & 89 HWY,DANBURY,NC,27016,STOKES,49180.0,U,CAH,...,640268.0,622592.0,17676.0,1.154767,578366.0,801800.0,159516.0,702117.0,601721,1.0
4,110003,MAYO CLINIC HEALTH SYSTEM WAYCROSS,1900 TEBEAU STREET,WAYCROSS,GA,31501,WARE,99911.0,R,STH,...,-6170184.0,NaN,-6170184.0,0.271734,3113423.0,11231324.0,1331.0,8211.0,603846,1.0


In [182]:
mainFile2 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])
mainFile2.head()

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,CON,Urban
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181,1.0,1.0
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095,1.0,2.0
2,450178,PECOS COUNTY MEMORIAL HOSPITAL,386 W IH-10,FORT STOCKTON,TX,79735,PECOS,99945.0,R,STH,...,NaN,576278.0,0.455741,719760.0,1275211.0,NaN,NaN,601392,0.0,2.0
3,341317,PIONEER HLTH SERV OF STOKES CNTY,1570 NC & 89 HWY,DANBURY,NC,27016,STOKES,49180.0,U,CAH,...,622592.0,17676.0,1.154767,578366.0,801800.0,159516.0,702117.0,601721,1.0,1.0
4,110003,MAYO CLINIC HEALTH SYSTEM WAYCROSS,1900 TEBEAU STREET,WAYCROSS,GA,31501,WARE,99911.0,R,STH,...,NaN,-6170184.0,0.271734,3113423.0,11231324.0,1331.0,8211.0,603846,1.0,2.0


In [183]:
#Append lattitude and longitude using google maps api

In [184]:
#Append calculated field profit percent to file
profit = mainFile2["Net Income"]/mainFile2["Gross Revenue"]
mainFile2["Profit %"] = profit
#mainFile2.head()

In [185]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient
#mainFile2.head()

In [186]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient
#mainFile2.head()

In [187]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Gross Revenue"]
mainFile2["Charity %"] = charity
#mainFile2.head()

In [188]:
#Is there a way to replace blank lines with 0?

In [189]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Gross Revenue"]
mainFile2["Uncompensated %"] = charity
#mainFile2.head()

In [190]:
#Is there a way to replace blank lines with 0?

In [250]:
bins = [-100, -.01, .01, 100]
group_names = ["3", "2", "1"]

In [251]:
mainFile2["Profit Score"] = pd.cut(mainFile2["Profit %"], bins, labels=group_names, include_lowest=True)
mainFile2.head()

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Stand-Alone SCHIP Charges,rpt_rec_num,CON,Urban,Profit %,Inpatient %,Outpatient %,Charity %,Uncompensated %,Profit Score
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,NaN,599181,1.0,1.0,0.122992,0.311881,0.688119,0.001229,0.001229,1
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,NaN,601095,1.0,2.0,-0.048746,0.325730,0.674270,0.000156,0.000156,3
2,450178,PECOS COUNTY MEMORIAL HOSPITAL,386 W IH-10,FORT STOCKTON,TX,79735,PECOS,99945.0,R,STH,...,NaN,601392,0.0,2.0,0.068136,0.240965,0.759035,NaN,NaN,1
3,341317,PIONEER HLTH SERV OF STOKES CNTY,1570 NC & 89 HWY,DANBURY,NC,27016,STOKES,49180.0,U,CAH,...,702117.0,601721,1.0,1.0,0.002345,0.275801,0.724199,NaN,NaN,2
4,110003,MAYO CLINIC HEALTH SYSTEM WAYCROSS,1900 TEBEAU STREET,WAYCROSS,GA,31501,WARE,99911.0,R,STH,...,8211.0,603846,1.0,2.0,-0.037184,0.391293,0.608707,0.004715,0.004715,3


In [252]:
#Targeting the dataset on columns (metrics) important to effort
mainFile3 = mainFile2[['Profit Score','Profit %','CON', 'Urban','Provider Type','Type of Control','FTE - Employees on Payroll', 'Number of Beds','Total Discharges (V + XVIII + XIX + Unknown)','Cost of Charity Care','Cost of Uncompensated Care','Total Costs', 'Total fixed Assets', 'Total Long Term Liabilities', 'Inpatient Revenue', 'Outpatient Revenue', 'Gross Revenue', 'Inpatient %', 'Outpatient %', 'Charity %', 'Uncompensated %']]
mainFile3.head()
#Excluded Wage Related costs because the high number of unavailable metrics

,Profit Score,Profit %,CON,Urban,Provider Type,Type of Control,FTE - Employees on Payroll,Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Cost of Charity Care,...,Total Costs,Total fixed Assets,Total Long Term Liabilities,Inpatient Revenue,Outpatient Revenue,Gross Revenue,Inpatient %,Outpatient %,Charity %,Uncompensated %
0,1,0.122992,1.0,1.0,1,2,322.10,70.0,435.0,26358.0,...,7704339.0,9205205.0,5012389.0,6691433.0,14763631.0,21455064.0,0.311881,0.688119,0.001229,0.001229
1,3,-0.048746,1.0,2.0,1,4,27.24,64.0,196.0,3556.0,...,3958521.0,9992010.0,NaN,7448346.0,15418247.0,22866593.0,0.325730,0.674270,0.000156,0.000156
2,1,0.068136,0.0,2.0,1,10,149.17,25.0,104.0,NaN,...,3854574.0,15925932.0,19689949.0,2038040.0,6419770.0,8457810.0,0.240965,0.759035,NaN,NaN
3,2,0.002345,1.0,1.0,1,4,112.17,25.0,52.0,NaN,...,5382132.0,4216311.0,4511968.0,2078831.0,5458594.0,7537425.0,0.275801,0.724199,NaN,NaN
4,3,-0.037184,1.0,2.0,1,9,923.39,155.0,2110.0,782377.0,...,39198497.0,61187548.0,672577.0,64929188.0,101005874.0,165935062.0,0.391293,0.608707,0.004715,0.004715


In [253]:
#Output geocoded file to eliminate multiple hits on googlemaps
mainFile3.to_csv("2017targetedmain.csv", index=False, header=True)

In [254]:
# Identify incomplete rows
mainFile3.count()

Profit Score                                    5888
Profit %                                        5888
CON                                             6168
Urban                                           6090
Provider Type                                   6171
Type of Control                                 6171
FTE - Employees on Payroll                      6063
Number of Beds                                  6085
Total Discharges (V + XVIII + XIX + Unknown)    6077
Cost of Charity Care                            4376
Cost of Uncompensated Care                      4707
Total Costs                                     6090
Total fixed Assets                              5812
Total Long Term Liabilities                     4963
Inpatient Revenue                               5889
Outpatient Revenue                              5386
Gross Revenue                                   5893
Inpatient %                                     5889
Outpatient %                                  

In [255]:
# Drop all rows with missing information
mainFile4 = mainFile3.dropna(how='any')

In [256]:
# Identify incomplete rows
mainFile4.count()

Profit Score                                    3816
Profit %                                        3816
CON                                             3816
Urban                                           3816
Provider Type                                   3816
Type of Control                                 3816
FTE - Employees on Payroll                      3816
Number of Beds                                  3816
Total Discharges (V + XVIII + XIX + Unknown)    3816
Cost of Charity Care                            3816
Cost of Uncompensated Care                      3816
Total Costs                                     3816
Total fixed Assets                              3816
Total Long Term Liabilities                     3816
Inpatient Revenue                               3816
Outpatient Revenue                              3816
Gross Revenue                                   3816
Inpatient %                                     3816
Outpatient %                                  

In [257]:
# Select features to be used as x values
features1 = mainFile4[['CON', 'Urban','Provider Type','Type of Control','FTE - Employees on Payroll', 'Number of Beds','Total Discharges (V + XVIII + XIX + Unknown)','Cost of Charity Care','Cost of Uncompensated Care','Total Costs', 'Total fixed Assets', 'Total Long Term Liabilities', 'Inpatient Revenue', 'Outpatient Revenue', 'Gross Revenue', 'Inpatient %', 'Outpatient %', 'Charity %', 'Uncompensated %']]
features1.head()

,CON,Urban,Provider Type,Type of Control,FTE - Employees on Payroll,Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Cost of Charity Care,Cost of Uncompensated Care,Total Costs,Total fixed Assets,Total Long Term Liabilities,Inpatient Revenue,Outpatient Revenue,Gross Revenue,Inpatient %,Outpatient %,Charity %,Uncompensated %
0,1.0,1.0,1,2,322.10,70.0,435.0,26358.0,284986.0,7704339.0,9205205.0,5012389.0,6691433.0,14763631.0,21455064.0,0.311881,0.688119,0.001229,0.001229
4,1.0,2.0,1,9,923.39,155.0,2110.0,782377.0,3793100.0,39198497.0,61187548.0,672577.0,64929188.0,101005874.0,165935062.0,0.391293,0.608707,0.004715,0.004715
6,1.0,2.0,1,2,378.26,49.0,632.0,14670.0,269451.0,20033059.0,65893849.0,34152807.0,16852176.0,44004253.0,60856429.0,0.276917,0.723083,0.000241,0.000241
7,1.0,1.0,1,2,693.11,48.0,1422.0,1989303.0,1882929.0,43205601.0,-1046227.0,11247.0,39423493.0,74054548.0,113478041.0,0.347411,0.652589,0.017530,0.017530
8,1.0,1.0,1,2,1452.05,268.0,9188.0,9740925.0,15188401.0,222474406.0,177612000.0,125362000.0,358393348.0,625072412.0,983465760.0,0.364419,0.635581,0.009905,0.009905


In [258]:
#Output features file as a reference document
features1.to_csv("2017features.csv", index=False, header=True)

In [259]:
# Identify the outcome column/data
y = mainFile4[['Profit %']]
y.head()

,Profit %
0,0.122992
4,-0.037184
6,-0.090105
7,-0.021699
8,0.018523


In [260]:
y.describe()

,Profit %
count,3816.000000
mean,0.012163
std,0.060865
min,-0.726599
25%,-0.006688
50%,0.012185
75%,0.032615
max,0.767821


In [261]:
# Identify the outcome column/data
yy = mainFile4[['Profit Score']]
yy.head()

,Profit Score
0,1
4,3
6,3
7,3
8,1


In [262]:
yy.describe()

,Profit Score
count,3816
unique,3
top,1
freq,2007


In [200]:
y.shape

(3816, 1)

In [201]:
X = features1
X.shape

(3816, 19)

In [203]:
# model = LinearRegression()
# model.fit(X, y)
# score = model.score(X, y)
# print(f"R2 Score: {score}")

In [164]:
#Create a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [165]:
y_train.shape

(2862, 1)

In [166]:
X_train.shape

(2862, 19)

In [167]:
y_test.shape

(954, 1)

In [168]:
X_test.shape

(954, 19)

In [169]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [170]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=15, step=1)
selector = selector.fit(X_train_scaled, y_train)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: Unknown label type: 'continuous'

In [ ]:
selector.support_

In [ ]:
selector.ranking_